In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Resizing, Lambda, GlobalAveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

In [ ]:
def load_data(dataset_path, target_size=(299, 299)):
    X, y = [], []
    class_labels = sorted(os.listdir(dataset_path))  # Extract labels from folder names
    print(f"Detected classes: {class_labels}")
    
    for label in class_labels:
        label_path = os.path.join(dataset_path, label)
        if not os.path.isdir(label_path):
            continue
        
        for file in os.listdir(label_path):
            file_path = os.path.join(label_path, file)
            try:
                img = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)  # Load as grayscale
                img = cv2.resize(img, target_size)  # Resize to match model input
                img = np.stack([img] * 3, axis=-1)  # Convert to 3 channels
                X.append(img)
                y.append(label)
            except Exception as e:
                print(f"Error loading {file_path}: {e}")
    
    X = np.array(X) / 255.0  # Normalize pixel values
    y = np.array(y)
    return X, y

In [ ]:
def preprocess_labels(y):
    le = LabelEncoder()
    y_encoded = le.fit_transform(y)
    y_onehot = to_categorical(y_encoded)
    return y_onehot, le

In [ ]:
def scheduler(epoch, lr):
    if epoch > 0 and epoch % 100 == 0:
        return lr * 0.5
    return lr

In [ ]:
def build_model(input_shape, num_classes):
    model = Sequential()
    model.add(Resizing(299, 299, input_shape=input_shape))
    model.add(Lambda(tf.keras.applications.inception_v3.preprocess_input))
    
    base_model = tf.keras.applications.InceptionV3(
        weights='imagenet', include_top=False, input_shape=(299, 299, 3)
    )
    for layer in base_model.layers:
        layer.trainable = False
    model.add(base_model)
    model.add(GlobalAveragePooling2D())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))
    return model

In [ ]:
def main():
    dataset_path = 'RAVDESS_mel_spectrograms'  # Update with your dataset directory
    X, y = load_data(dataset_path)
    
    y_onehot, le = preprocess_labels(y)
    print("Detected labels:", le.classes_)
    
    X_train, X_val, y_train, y_val = train_test_split(
        X, y_onehot, test_size=0.2, random_state=42, stratify=np.argmax(y_onehot, axis=1)
    )
    
    input_shape = X_train.shape[1:]
    num_classes = y_onehot.shape[1]
    model = build_model(input_shape, num_classes)
    
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model.summary()
    
    early_stop = EarlyStopping(monitor='val_loss', patience=200, restore_best_weights=True)
    lr_scheduler = LearningRateScheduler(scheduler)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=50, min_lr=1e-6)
    
    history = model.fit(
        X_train, y_train, epochs=500, batch_size=32, validation_data=(X_val, y_val),
        callbacks=[early_stop, lr_scheduler, reduce_lr]
    )
    
    loss, acc = model.evaluate(X_val, y_val)
    print(f"Validation Loss: {loss:.4f}, Validation Accuracy: {acc:.4f}")
    
    # model.save('sentiment_inception_model.h5')
    # print("Model saved as 'sentiment_inception_model.h5'")

In [ ]:
if __name__ == '__main__':
    main()

Detected classes: ['angry', 'calm', 'disgust', 'fearful', 'happy', 'neutral', 'sad', 'surprised']
Detected labels: ['angry' 'calm' 'disgust' 'fearful' 'happy' 'neutral' 'sad' 'surprised']
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resizing (Resizing)         (None, 299, 299, 3)       0         
                                                                 
 lambda (Lambda)             (None, 299, 299, 3)       0         
                                                                 
 inception_v3 (Functional)   (None, 8, 8, 2048)        21802784  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 256)               524544    
